In [ ]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.applications import MobileNetV2
from keras.models import Model
from keras.layers import Dense, Dropout, Flatten, GlobalAveragePooling2D
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping

print("TensorFlow Version:", tf.__version__)

TensorFlow Version: 2.13.0


In [ ]:
train_dir = "archive/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train"

In [ ]:
train_dir = "archive/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train"
IMG_SIZE = (128, 128)
BATCH_SIZE = 16

datagen = ImageDataGenerator(
    rescale=1/255.0,
    validation_split=0.15,
    rotation_range=10,      
    horizontal_flip=True
)

train_gen = datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training'
)

val_gen = datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'
)

num_classes = train_gen.num_classes
print("Number of Classes =", num_classes)

base_model = MobileNetV2(
    weights="imagenet",
    include_top=False,
    input_shape=(128, 128, 3)
)

for layer in base_model.layers[:-20]:
    layer.trainable = False
for layer in base_model.layers[-20:]:
    layer.trainable = True

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x)
output = Dense(num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output)

model.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

model.summary()

checkpoint = ModelCheckpoint(
    "best_mobilenet_model.h5",
    monitor="val_accuracy",
    save_best_only=True,
    mode="max",
    verbose=1
)

early_stop = EarlyStopping(
    monitor="val_loss",
    patience=5,
    restore_best_weights=True
)

EPOCHS = 10

history = model.fit(
    train_gen,
    epochs=EPOCHS,
    validation_data=val_gen,
    callbacks=[checkpoint, early_stop]
)

model.save("plant_disease_mobilenetv2.h5")
print("Model saved as plant_disease_mobilenetv2.h5")


Found 59766 images belonging to 38 classes.
Found 10529 images belonging to 38 classes.
Number of Classes = 38
9406464/9406464 [==============================] - 257s 27us/step
Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 128, 128, 3)]        0         []                            
                                                                                                  
 Conv1 (Conv2D)              (None, 64, 64, 32)           864       ['input_2[0][0]']             
                                                                                                  
 bn_Conv1 (BatchNormalizati  (None, 64, 64, 32)           128       ['Conv1[0][0]']               
 on)                                                                                              
              

d:\uni\projects\plant_disease_detection\venv\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


3736/3736 [==============================] - 1302s 347ms/step - loss: 0.5137 - accuracy: 0.8482 - val_loss: 0.2161 - val_accuracy: 0.9274
Epoch 2/10
3736/3736 [==============================] - ETA: 0s - loss: 0.2011 - accuracy: 0.9359
Epoch 2: val_accuracy improved from 0.92744 to 0.95555, saving model to best_mobilenet_model.h5
3736/3736 [==============================] - 1443s 386ms/step - loss: 0.2011 - accuracy: 0.9359 - val_loss: 0.1358 - val_accuracy: 0.9556
Epoch 3/10
3736/3736 [==============================] - ETA: 0s - loss: 0.1492 - accuracy: 0.9514
Epoch 3: val_accuracy did not improve from 0.95555
3736/3736 [==============================] - 1428s 382ms/step - loss: 0.1492 - accuracy: 0.9514 - val_loss: 0.1416 - val_accuracy: 0.9546
Epoch 4/10
3736/3736 [==============================] - ETA: 0s - loss: 0.1152 - accuracy: 0.9622
Epoch 4: val_accuracy did not improve from 0.95555
3736/3736 [==============================] - 1549s 415ms/step - loss: 0.1152 - accuracy: 0.962